# This part 3 of 3 is TBD very soon <a class='tocSkip'>

**Previous notebooks:** <br>
[Part 1: Data mining and munging](tdp_1_3_data_mining_and_cleaning.ipynb)<br>
[Part 2: Data stats and visuals](tdp_2_3_data_statistics_visualization.ipynb)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns; sns.set()

### Data loading <a class='tocSkip'>

In [2]:
# Main data for modeling
biker_class = pd.read_csv('processed/clean_biker_class.csv', index_col=0)

# # Additional parts for statistics
# biker_stats = pd.read_csv('preprocessed/pre_biker_stats.csv', index_col=0)
# biker_by_stage = pd.read_csv('preprocessed/pre_biker_by_stage.csv', index_col=0)
# stage_results = pd.read_csv('preprocessed/pre_stages.csv', index_col=False)
# stage_class = pd.read_csv('preprocessed/pre_stages_summary.csv', index_col=False)

In [3]:
# Fixing 'date' type column
biker_class['date'] = pd.to_datetime(biker_class['date'])

# Preparing data

In [4]:
# Baseline prediction for 0s
1 - biker_class.y_label.mean()

0.9352997145575642

In [5]:
# Poping out label column
y = biker_class.pop('y_label')

In [6]:
y[6:15]

6     1
7     1
8     1
9     1
10    0
11    0
12    0
13    0
14    0
Name: y_label, dtype: int64

-------

In [7]:
biker_class.columns

Index(['name', 'nation', 'age', 'team', 'year', 'stage', 'position',
       'time_in_s', 'gc', 'gc_time_in_s', 'mount_jerseys', 'point_jerseys',
       'sprint_jerseys', 'youth_jerseys', 'followers', 'total_wins', 'n_tours',
       'n_stages', 'avg_year', 'best_pos_ovr', 'avg_pos_ovr', 'worst_pos_ovr',
       'best_time_ovr', 'avg_time_ovr', 'worst_time_ovr', 'best_gc_ovr',
       'avg_gc_ovr', 'worst_gc_ovr', 'best_gc_time_ovr', 'avg_gc_time_ovr',
       'worst_gc_time_ovr', 'n_top10_ovr', 'n_win_ovr', 'best_pos_ovr_l3',
       'avg_pos_ovr_l3', 'worst_pos_ovr_l3', 'best_time_ovr_l3',
       'avg_time_ovr_l3', 'worst_time_ovr_l3', 'best_gc_ovr_l3',
       'avg_gc_ovr_l3', 'worst_gc_ovr_l3', 'best_gc_time_ovr_l3',
       'avg_gc_time_ovr_l3', 'worst_gc_time_ovr_l3', 'n_top10_l3', 'n_win_l3',
       'perc_top10_ovr', 'perc_win_ovr', '2015_div', '2015_fc_rank',
       '2015_race_days', '2015_team', '2015_wins', '2015_year_km', '2016_div',
       '2016_fc_rank', '2016_race_days', '2016_te

In [8]:
print(biker_class.shape)
biker_class.head()

(5255, 83)


,name,nation,age,team,year,stage,position,time_in_s,gc,gc_time_in_s,...,date,stage_type_code,dist,finish,temp,rel_humid,wind_dir,wind_speed,clouds_ovrl,visibility
0,Y.Hutarovich,Belarus,31,AG2R La Mondiale,2014,1,1,20870.0,1,20860.0,...,2014-08-03,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9
1,R.Maikin,Russia,24,RusVelo,2014,1,2,20870.0,2,20864.0,...,2014-08-03,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9
2,M.Mori,Italy,34,Lampre - Merida,2014,1,3,20870.0,3,20866.0,...,2014-08-03,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9
3,G.Boivin,Canada,25,Cannondale Pro Cycling,2014,1,4,20870.0,6,20870.0,...,2014-08-03,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9
4,M.Haller,Austria,23,Team Katusha,2014,1,5,20870.0,7,20870.0,...,2014-08-03,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9


## Remapping & Dropping

In [9]:
# Due to too many levels, trying with the '_team' data removed
# Try: change team levels for only 5 top + 1 as a rest
biker_class.drop('team', axis=1, inplace=True)
biker_class.drop(biker_class.loc[:,'2015_team':'2018_team':6], axis=1, inplace=True)

In [10]:
biker_class.loc[:,'2015_div':'2018_div':5].head()

,2015_div,2016_div,2017_div,2018_div
0,PROF,PROF,NONE,NONE
1,PROF,PROF,PROF,PROF
2,PRT,PRT,PRT,PRT
3,CONT,CONT,PROF,PROF
4,PRT,PRT,PRT,PRT


In [11]:
# Remapping '_div' team types with numbers as they show a true level of professionalism
biker_class.loc[:,'2015_div':'2018_div':5] = biker_class.loc[:,'2015_div':'2018_div':5]\
                                                        .replace({'PRT':1, 'PROF':2, 'CONT':3, 'NONE':4})

In [12]:
biker_class.loc[:,'2015_div':'2018_div':5].head()

,2015_div,2016_div,2017_div,2018_div
0,2,2,4,4
1,2,2,2,2
2,1,1,1,1
3,3,3,2,2
4,1,1,1,1


## Information about features

### Datetime feature (1)

In [13]:
biker_class.select_dtypes('datetime').head()

,date
0,2014-08-03
1,2014-08-03
2,2014-08-03
3,2014-08-03
4,2014-08-03


### Categorical features (4)

In [14]:
biker_class.select_dtypes('object').columns

Index(['name', 'nation', 'finish', 'wind_dir'], dtype='object')

In [15]:
print(biker_class.select_dtypes('object').shape)
biker_class.select_dtypes('object').head(3)

(5255, 4)


,name,nation,finish,wind_dir
0,Y.Hutarovich,Belarus,Bygdoszcz,ESE
1,R.Maikin,Russia,Bygdoszcz,ESE
2,M.Mori,Italy,Bygdoszcz,ESE


### Numeric features (73)

In [16]:
biker_class.select_dtypes('number').columns

Index(['age', 'year', 'stage', 'position', 'time_in_s', 'gc', 'gc_time_in_s',
       'mount_jerseys', 'point_jerseys', 'sprint_jerseys', 'youth_jerseys',
       'followers', 'total_wins', 'n_tours', 'n_stages', 'avg_year',
       'best_pos_ovr', 'avg_pos_ovr', 'worst_pos_ovr', 'best_time_ovr',
       'avg_time_ovr', 'worst_time_ovr', 'best_gc_ovr', 'avg_gc_ovr',
       'worst_gc_ovr', 'best_gc_time_ovr', 'avg_gc_time_ovr',
       'worst_gc_time_ovr', 'n_top10_ovr', 'n_win_ovr', 'best_pos_ovr_l3',
       'avg_pos_ovr_l3', 'worst_pos_ovr_l3', 'best_time_ovr_l3',
       'avg_time_ovr_l3', 'worst_time_ovr_l3', 'best_gc_ovr_l3',
       'avg_gc_ovr_l3', 'worst_gc_ovr_l3', 'best_gc_time_ovr_l3',
       'avg_gc_time_ovr_l3', 'worst_gc_time_ovr_l3', 'n_top10_l3', 'n_win_l3',
       'perc_top10_ovr', 'perc_win_ovr', '2015_div', '2015_fc_rank',
       '2015_race_days', '2015_wins', '2015_year_km', '2016_div',
       '2016_fc_rank', '2016_race_days', '2016_wins', '2016_year_km',
       '2017_div',

In [17]:
print(biker_class.select_dtypes('number').shape)
biker_class.select_dtypes('number').head()

(5255, 73)


,age,year,stage,position,time_in_s,gc,gc_time_in_s,mount_jerseys,point_jerseys,sprint_jerseys,...,2018_race_days,2018_wins,2018_year_km,stage_type_code,dist,temp,rel_humid,wind_speed,clouds_ovrl,visibility
0,31,2014,1,1,20870.0,1,20860.0,0,6,0,...,0.0,0.0,0.0,1,226.0,31.5,38,2,6,9
1,24,2014,1,2,20870.0,2,20864.0,0,2,0,...,44.0,0.0,7349.7,1,226.0,31.5,38,2,6,9
2,34,2014,1,3,20870.0,3,20866.0,0,0,0,...,65.0,0.0,10322.1,1,226.0,31.5,38,2,6,9
3,25,2014,1,4,20870.0,6,20870.0,0,1,0,...,63.0,0.0,10485.0,1,226.0,31.5,38,2,6,9
4,23,2014,1,5,20870.0,7,20870.0,1,0,0,...,17.0,0.0,2733.3,1,226.0,31.5,38,2,6,9


## Transforming features into dummy categories & scaling values

In [18]:
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [19]:
data_numerics = biker_class.select_dtypes(include = ['number'])
# Skipping 'name' column
data_categorical = biker_class.select_dtypes(exclude = ['number']).iloc[:,1:]

categorical_dummies = pd.get_dummies(data_categorical)

numeric_scaler = StandardScaler()
numerics_scaled = numeric_scaler.fit_transform(data_numerics)
numerics_scaled = pd.DataFrame(numerics_scaled, columns=data_numerics.columns, index=data_numerics.index)

data_adult = pd.concat([biker_class.name, categorical_dummies, numerics_scaled], axis=1)

C:\Users\cyberdeck\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\cyberdeck\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [20]:
print(data_adult.shape)
data_adult.head()

(5255, 152)


,name,date,nation_Algeria,nation_Argentina,nation_Australia,nation_Austria,nation_Belarus,nation_Belgium,nation_Brazil,nation_Canada,...,2018_race_days,2018_wins,2018_year_km,stage_type_code,dist,temp,rel_humid,wind_speed,clouds_ovrl,visibility
0,Y.Hutarovich,2014-08-03,0,0,0,0,1,0,0,0,...,-1.987928,-0.404551,-1.951352,-0.73624,1.13983,1.405869,-0.032021,-0.443167,0.614099,1.247777
1,R.Maikin,2014-08-03,0,0,0,0,0,0,0,0,...,-0.165444,-0.404551,-0.011144,-0.73624,1.13983,1.405869,-0.032021,-0.443167,0.614099,1.247777
2,M.Mori,2014-08-03,0,0,0,0,0,0,0,0,...,0.704378,-0.404551,0.773524,-0.73624,1.13983,1.405869,-0.032021,-0.443167,0.614099,1.247777
3,G.Boivin,2014-08-03,0,0,0,0,0,0,0,1,...,0.621538,-0.404551,0.816527,-0.73624,1.13983,1.405869,-0.032021,-0.443167,0.614099,1.247777
4,M.Haller,2014-08-03,0,0,0,1,0,0,0,0,...,-1.283787,-0.404551,-1.229803,-0.73624,1.13983,1.405869,-0.032021,-0.443167,0.614099,1.247777


## Modeling

In [21]:
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from scipy.stats.distributions import uniform, randint

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [22]:
# Baseline prediction for 0s
1 - y.mean()

0.9352997145575642

In [23]:
# Spliting data depending on the date column (not random), because going down the dataframe it is 'sequencial' data,
# year by year, stage by stage

# Spliting observations to prevent data leakage
# Split made after last stage of year 2017
cutoff = 3120

X_train = data_adult.iloc[:cutoff,:]
X_test = data_adult.iloc[cutoff:,:]

y_train = y[:cutoff]
y_test = y[cutoff:] 

In [24]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(3120, 152) (2135, 152)
(3120,) (2135,)


In [25]:
print('Train batch ------------')
print(y_train.value_counts())
print()
print('Test batch -------------')
print(y_test.value_counts())

Train batch ------------
0    2930
1     190
Name: y_label, dtype: int64

Test batch -------------
0    1985
1     150
Name: y_label, dtype: int64


### Baseline

In [26]:
X_train_cut = X_train.iloc[:,2:]
X_test_cut = X_test.iloc[:,2:]

#### Support Vector Machines

In [27]:
svc = SVC(class_weight='balanced', probability=True, random_state=42, gamma='auto')
scores = cross_val_score(X=X_train_cut,
                         y=y_train,
                         estimator=svc,
                         cv=StratifiedKFold(10, shuffle=True),
                         scoring='roc_auc',
                         n_jobs=-1)

In [28]:
svc.fit(X_train_cut, y_train)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [29]:
scores.mean()

0.91990299982037

In [30]:
print(classification_report(svc.predict(X_test_cut), y_test))

              precision    recall  f1-score   support

           0       0.85      0.99      0.91      1715
           1       0.84      0.30      0.44       420

   micro avg       0.85      0.85      0.85      2135
   macro avg       0.85      0.64      0.68      2135
weighted avg       0.85      0.85      0.82      2135



In [31]:
svc.predict_proba(X_test_cut)

array([[9.99943846e-01, 5.61541064e-05],
       [9.80404435e-01, 1.95955649e-02],
       [9.31844831e-01, 6.81551688e-02],
       ...,
       [9.99852727e-01, 1.47273272e-04],
       [9.99927605e-01, 7.23952728e-05],
       [9.98421677e-01, 1.57832350e-03]])

Customizing parameters

In [32]:
param_grid = {'kernel':['rbf'],
               'gamma':uniform(0.0,0.5),
               'C':uniform(0.25,10.0-0.25)}
#               ,
#               {'kernel':['poly'],
#                'degree':randint(1,7),
#                'C':uniform(0.25,10.0-0.25)}
#               ,
#               {'kernel':['sigmoid'],
#                'C':uniform(0.25,10.0-0.25)}]

In [33]:
grid = RandomizedSearchCV(
                        n_iter=30, 
                        estimator=svc, 
                        cv=StratifiedKFold(10, shuffle=True), 
                        param_distributions=param_grid,
                        scoring='roc_auc',
                        n_jobs=-1
                        )

In [34]:
grid.fit(X_train_cut, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=True),
          error_score='raise-deprecating',
          estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=42, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'kernel': ['rbf'], 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020AD96D7780>, 'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020AD96F0F98>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [35]:
grid.best_params_

{'C': 2.6643624401904624, 'gamma': 0.00901864612920139, 'kernel': 'rbf'}

In [36]:
grid.predict_proba(X_test_cut)

array([[9.99972171e-01, 2.78287334e-05],
       [9.34483972e-01, 6.55160277e-02],
       [9.45483210e-01, 5.45167902e-02],
       ...,
       [9.99839286e-01, 1.60713899e-04],
       [9.99989480e-01, 1.05195848e-05],
       [9.98282562e-01, 1.71743772e-03]])

In [37]:
# After customizing params
grid.best_score_

0.9271780132926172

In [38]:
print(classification_report(grid.predict(X_test_cut), y_test))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      1860
           1       0.70      0.38      0.49       275

   micro avg       0.90      0.90      0.90      2135
   macro avg       0.81      0.68      0.72      2135
weighted avg       0.89      0.90      0.89      2135



#### Logistic Regression

Baseline

In [39]:
lr = LogisticRegression(class_weight='balanced', random_state=42, solver='liblinear')
scores = cross_val_score(X=X_train_cut,
                         y=y_train,
                         estimator=lr,
                         cv=StratifiedKFold(10, shuffle=True),
                         scoring='roc_auc',
                         n_jobs=-1)
lr.fit(X_train_cut, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [40]:
scores.mean()

0.9813005209268907

In [41]:
print(classification_report(lr.predict(X_test_cut), y_test))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1795
           1       0.99      0.44      0.61       340

   micro avg       0.91      0.91      0.91      2135
   macro avg       0.95      0.72      0.78      2135
weighted avg       0.92      0.91      0.89      2135



Customizing parameters

In [42]:
param_grid = {'penalty':['l1','l2'],
              'C':uniform(0.25,10.0-0.25)}

In [43]:
grid = RandomizedSearchCV(
                        n_iter=40, 
                        estimator=lr, 
                        cv=StratifiedKFold(10, shuffle=True), 
                        param_distributions=param_grid,
                        scoring='roc_auc',
                        n_jobs=-1
                        )

In [44]:
grid.fit(X_train_cut, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=True),
          error_score='raise-deprecating',
          estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=40, n_jobs=-1,
          param_distributions={'penalty': ['l1', 'l2'], 'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020AD9C2F208>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [45]:
grid.best_params_

{'C': 5.811565552444372, 'penalty': 'l1'}

In [46]:
grid.best_score_

0.9986527752829173

In [47]:
print(classification_report(grid.predict(X_test_cut), y_test))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1967
           1       0.99      0.88      0.93       168

   micro avg       0.99      0.99      0.99      2135
   macro avg       0.99      0.94      0.96      2135
weighted avg       0.99      0.99      0.99      2135



### Features engineering

First feature engineering was carried out above using L1 regularization with the Logistic Regression

In [48]:
data_adult_cut = data_adult.iloc[:,2:]

**Variance Threshold**

In [49]:
from sklearn.feature_selection import VarianceThreshold

In [50]:
var = VarianceThreshold(threshold=0.1)
var.fit(data_adult_cut)
data_adult_var = var.transform(data_adult_cut)

In [51]:
data_adult_cut.shape, data_adult_var.shape

((5255, 150), (5255, 77))

In [52]:
X_train_var = data_adult_var[:cutoff]
X_test_var = data_adult_var[cutoff:]

In [53]:
X_train_cut.shape, X_train_var.shape

((3120, 150), (3120, 77))

In [54]:
param_grid = {'penalty':['l1','l2'],
              'C':uniform(0.25,10.0-0.25)}

grid = RandomizedSearchCV(
                        n_iter=30, 
                        estimator=lr, 
                        cv=StratifiedKFold(10, shuffle=True), 
                        param_distributions=param_grid,
                        scoring='roc_auc',
                        n_jobs=-1
                        )

grid.fit(X_train_var, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=True),
          error_score='raise-deprecating',
          estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'penalty': ['l1', 'l2'], 'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020ADA4DA908>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [55]:
grid.best_params_

{'C': 4.605137955926278, 'penalty': 'l1'}

In [56]:
grid.best_score_

0.9985629603017783

In [57]:
print(classification_report(grid.predict(X_test_var), y_test))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1959
           1       0.99      0.85      0.91       176

   micro avg       0.99      0.99      0.99      2135
   macro avg       0.99      0.92      0.95      2135
weighted avg       0.99      0.99      0.99      2135



**Statistical tests**

In [58]:
#TBD: chi^2, ANOVA

**RFE**

In [59]:
from sklearn.feature_selection import RFE, RFECV

In [60]:
rfe_estimator = LogisticRegression(C=3.7, penalty='l1', class_weight='balanced', solver='liblinear')
# Takes very long time to compute RFECV with StratifiedKfold
# rfe_selector = RFECV(rfe_estimator, step=5, cv=StratifiedKFold(5, shuffle=True))
rfe_selector = RFE(rfe_estimator, step=1)

rfe_selector.fit(data_adult_cut, y)

RFE(estimator=LogisticRegression(C=3.7, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
  n_features_to_select=None, step=1, verbose=0)

In [61]:
data_adult_cut.columns[rfe_selector.get_support()]

Index(['nation_Colombia', 'nation_Ecuador', 'nation_Eritrea', 'nation_Estonia',
       'nation_Finland', 'nation_France', 'nation_Italy', 'nation_Luxembourg',
       'nation_Norway', 'nation_Poland', 'nation_Russia', 'nation_Sweden',
       'nation_USA', 'finish_Bukowina Tatrzanska', 'finish_Bygdoszcz',
       'finish_Katowice', 'finish_Krakow', 'finish_Kraków', 'finish_Nowy Sącz',
       'finish_Warsaw', 'finish_Warszawa', 'finish_Zabrze', 'finish_Zakopane',
       'wind_dir_  N', 'wind_dir_  S', 'wind_dir_ NW', 'wind_dir_ESE',
       'wind_dir_SSW', 'wind_dir_WSW', 'age', 'year', 'stage', 'position',
       'gc', 'mount_jerseys', 'point_jerseys', 'sprint_jerseys',
       'youth_jerseys', 'followers', 'total_wins', 'n_tours', 'n_stages',
       'best_pos_ovr', 'avg_pos_ovr', 'avg_time_ovr', 'best_gc_ovr',
       'worst_gc_ovr', 'n_top10_ovr', 'n_win_ovr', 'best_gc_ovr_l3',
       'best_gc_time_ovr_l3', 'avg_gc_time_ovr_l3', 'n_top10_l3', 'n_win_l3',
       'perc_top10_ovr', 'perc_win_

In [62]:
support_columns = data_adult_cut.columns[rfe_selector.get_support()]
data_adult_rfe = data_adult_cut[support_columns]

In [63]:
X_train_rfe = data_adult_rfe.iloc[:cutoff,:]
X_test_rfe = data_adult_rfe.iloc[cutoff:,:]

In [64]:
X_train_rfe.shape

(3120, 75)

In [65]:
param_grid = {'penalty':['l1','l2'],
              'C':uniform(0.25,10.0-0.25)}

grid = RandomizedSearchCV(
                        n_iter=30, 
                        estimator=lr, 
                        cv=StratifiedKFold(10, shuffle=True), 
                        param_distributions=param_grid,
                        scoring='roc_auc',
                        n_jobs=-1
                        )

grid.fit(X_train_rfe, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=True),
          error_score='raise-deprecating',
          estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'penalty': ['l1', 'l2'], 'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020ADA4AE320>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [66]:
grid.best_params_

{'C': 5.688419967799207, 'penalty': 'l1'}

In [67]:
grid.best_score_

0.9984551823244118

In [68]:
print(classification_report(grid.predict(X_test_rfe), y_test))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1965
           1       0.99      0.87      0.93       170

   micro avg       0.99      0.99      0.99      2135
   macro avg       0.99      0.93      0.96      2135
weighted avg       0.99      0.99      0.99      2135

